## Overview
- This notebook overviews the `Factory` classes used to run our experiments.
- These classes are intended to assist with book keeping by the recording what objects were used and how.
- By the end of this tutorial, you should hopefully understand how the following example works.

In [1]:
import os, sys  # enable relative imports
module_path = os.path.abspath(os.path.join('../experiments'))
if module_path not in sys.path:
    sys.path.append(module_path)    

import tensorflow as tf
from factories import default_factory_mode, FactoryManager

gpflow = FactoryManager("gpflow")
with default_factory_mode.ctx(True):
    model = gpflow.models.GPR(
        kernel=gpflow.kernels.Matern52(lengthscales=0.5), 
        likelihood=gpflow.likelihoods.Gaussian(variance=1e-3),
    )

X = tf.random.uniform([3, 2])
Y = tf.random.uniform([3, 1])
print(f"1. {model}\n")  # easy to serialize!
print(f"2. {model(data=(X, Y))}")  # initalizes a model

### Factory
- A `Factory` stores an object (such as a class) along with its path relative to a parent module.
- `Factory` instances have fields `factory: bool | None = None` that control their behavior when called (explored below).
- `Factory.factory_ctx` is a context manager for conveniently switching between modes.
- The `default_factory_mode` setting dictates the behavior of `Factory` instances whose `factory` field is `None`.

### Loader
- A `Loader` is a `Factory` for an object, such as a type or pre-built instance of a class.
- When called outside of factory mode, a `Loader` returns wrapped object.
- When called in factory mode, a `Loader` returns a `Builder` for the wrapped object if it is callable and raises a `RuntimeError` otherwise.


In [2]:
from factories import Loader

loader = Loader(path="gpflow.kernels.Matern52")
print(f"1. {loader}")  # here, factory mode is False
print(f"2. {loader()}")  # returns the Matern52 type

with default_factory_mode.ctx(True):  # here, factory mode is True
    print(f"3. {loader(variance=2.0)}") # returns a Builder for a Matern52 instance

1. Loader(path='gpflow.kernels.Matern52', factory=None)
2. <class 'gpflow.kernels.stationaries.Matern52'>
3. Builder(path='gpflow.kernels.Matern52', factory=None, args=[], keywords={'variance': 2.0, 'lengthscales': 1.0})


## Builder
- A `Builder` is a `Factory` that wraps and executes a `Callable` similar to `functools.partial`.
- During initialization, a `Builder` extracts the default arguments of the wrapped object.
- When called, a `Builder` resolves runtime arguments in the same way as `functools.partial`.
- When called outside of factory mode, the updated arguments are used to call the wrapped object.
- When called in factory mode, the updated argument are instead returned as part of a new `Builder` for the wrapped object.

In [3]:
from factories import Builder

builder = Builder(path="gpflow.kernels.Matern52", keywords={"lengthscales": 0.5})
print(f"1. {builder}")
print(f"2. {builder(variance=3.0)}")  # kernel with variance 3

with default_factory_mode.ctx(True):
    new_builder = builder(variance=2.0)

print(f"3. {new_builder}")
print(f"4. {new_builder()}")  # kernel with variance 2

1. Builder(path='gpflow.kernels.Matern52', factory=None, args=[], keywords={'variance': 1.0, 'lengthscales': 0.5})
2. <gpflow.kernels.stationaries.Matern52 object at 0x110ed9510>
3. Builder(path='gpflow.kernels.Matern52', factory=None, args=[], keywords={'variance': 2.0, 'lengthscales': 0.5})
4. <gpflow.kernels.stationaries.Matern52 object at 0x110ec88d0>


## build
- The `build` helper function can be used to invoke factories and basic containers thereof.
- `build` supports the following container types: dictionaries, sequences, NamedTuples, and dataclasses.
- Note that `Builder` subclasses `Partial`. The only difference between these classes is that the latter is not invoked when calling `build`.

In [4]:
from typing import NamedTuple
from factories import build, Partial

class FactoryTuple(NamedTuple):
    loader: Loader  # invoked when calling `build`
    builder: Builder  # invoked when calling `build`
    partial: Partial   # not invoked when calling `build`

factories = FactoryTuple(loader, builder, builder.as_partial())
print(f"1. {factories}\n")
print(f"2. {build(factories)}")

1. FactoryTuple(loader=Loader(path='gpflow.kernels.Matern52', factory=None), builder=Builder(path='gpflow.kernels.Matern52', factory=None, args=[], keywords={}), partial=Partial(path='gpflow.kernels.Matern52', factory=None, args=[], keywords={'variance': 1.0, 'lengthscales': 0.5}))

2. FactoryTuple(loader=<class 'gpflow.kernels.stationaries.Matern52'>, builder=<gpflow.kernels.stationaries.Matern52 object at 0x110ed2c10>, partial=Partial(path='gpflow.kernels.Matern52', factory=None, args=[], keywords={'variance': 1.0, 'lengthscales': 0.5}))


### FactoryManager
- A `FactoryManager` is a convenience class for automatically creating `Loader` instances.
- A `FactoryManager` wraps a module and overwrites attibute access.
- When used outside of factory mode, attribute access returns the attributes of the wrapped module like normal.
- When used in factory mode, attribute access returns a `Loader` for the wrapped module's attributes.

In [5]:
from factories import default_factory_mode, FactoryManager

gpflow = FactoryManager("gpflow")
Kernel = gpflow.kernels.Matern52  # factory mode is False here
kernel = gpflow.kernels.Matern52(lengthscales=0.5)
print(f"1. {Kernel}")
print(f"2. {kernel}")

with default_factory_mode.ctx(True):  # factory mode is True here
    loader = gpflow.kernels.Matern52
    builder = loader(lengthscales=0.5)
    print(f"3. {loader}")
    print(f"4. {builder}")

1. <class 'gpflow.kernels.stationaries.Matern52'>
2. <gpflow.kernels.stationaries.Matern52 object at 0x110e9cc90>
3. Loader(path='gpflow.kernels.Matern52', factory=None)
4. Builder(path='gpflow.kernels.Matern52', factory=None, args=[], keywords={'variance': 1.0, 'lengthscales': 0.5})
